
Deliverable

Your task is to write a program that takes an input data set, applies the perceptron, and outputs the
final set of weights as well as a final confusion matrix.


In [12]:
#Import the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


A dataFrame is needed.

The dataFrame needs to be call 'df'

3 columns are needed: 'x1', 'x2' , 'y'


In [13]:

##########################################################################################
############################# DataFrame ##################################################
##########################################################################################

# Assign the dataframe to the df variable below

df


In [28]:
#Create weights W0
w=[0,0,0] #(w1,w2,w3)

#function that implemet the perceptron
def perceptron(x1_elem,x2_elem):
    pred_1=w[0] + w[1]*x1_elem + w[2]*x2_elem
    if(pred_1>=0):
        return (1)
    else:
        return (-1)

#Compares the expected result with the prediction. 
# 0: observations correctly predicted to be +1
# 1: observations incorrectly predicted to be +1
# 2: observations incorrectly predicted to be −1
# 3: observations correctly predicted to be −1
def eval_res(y_elem, predicted_elem):
    if(y_elem==1):
        if(predicted_elem==1):
            return (0)
        else:
            return (2)
    else:
        if(predicted_elem==1):
            return (1)
        else:
            return (3)

#If the prediction is correct return a True. if is misclassified return a False
def correct(result):
    if(result==0 or result==3): # Result is 0 if The correct value is 1 and predicted is 1. Result is 3 if The correct value is 0 and predicted is 0
        return (True)
    else:
        return (False)



--> Main Loop of the algorithm <--



In [30]:

historic_Acurracy=[]

for x in range(10000):
    
    #Execute the perceptron
    predicted = df.apply(lambda elem: perceptron(elem['x1'],elem['x2']),axis=1)
    #print(predicted)
    #Add the predicted values to the dataframe
    df["predicted"]=predicted
    #print(df)

    #Execute the evaluation and add it to the dataframe
    result = df.apply(lambda elem: eval_res(elem['y'],elem['predicted']),axis=1)
    df["result"]=result
    #print(df)

    #Count the number of each cases
    case_count=df.result.value_counts()
    #If an option does not appear we catch the exception, and we add a 0
    res=[]
    try:
        res.append(case_count.at[0])
    except KeyError:
        res.append(0)

    try:
        res.append(case_count.at[1])
    except KeyError:
        res.append(0)

    try:
        res.append(case_count.at[2])
    except KeyError:
        res.append(0)

    try:
        res.append(case_count.at[3])
    except KeyError:
        res.append(0)


    #Calculate the accuracy
    accuracy= (res[0]+res[3]) / (res[0]+res[1]+res[2]+res[3]) * 100
    
    if(x%50==0 or accuracy==100):
        print ("Loop Number: " + str(x))
        #Print the Confusion Matrix and the Accuracy
        print(res)
        print("|----------------|-----------------------|-----------------------|")
        print("|\t\t |\t Actual +1\t |\t Actual −1\t |")
        print("|----------------|-----------------------|-----------------------|")
        print("|Predicted +1 \t |\t " +str(res[0])+ "\t\t |\t " + str(res[1])+"\t\t |")
        print("|----------------|-----------------------|-----------------------|")
        print("|Predicted -1 \t |\t " +str(res[2])+ "\t\t |\t " + str(res[3])+"\t\t |")
        print("|----------------|-----------------------|-----------------------|")
        print("")
        print("Accuracy: " + str(accuracy) + " %")
    
    historic_Acurracy.append(accuracy)
    
    #If Accuracy is 100 % we exit the loop. If not we update the Weights
    if(accuracy==100):
        break
    else:
        #Aply the function to the dataframe
        correct_col=df.apply(lambda elem: correct(elem['result']),axis=1)
        df["correct"]=correct_col
        #print(df)

        #Pick One random Observation from the misclassified observations
        incorrect_Pred=df.loc[df['correct']==False]
        random_Obs=incorrect_Pred.sample()
        #random_Obs

        #Update the Weights
        elem=random_Obs.iloc[0]
        w[0]=w[0]+1*elem['y']
        w[1]=w[1]+elem['x1']*elem['y']
        w[2]=w[2]+elem['x2']*elem['y']
        
        if(x%50==0):
            print("weights:" + str(w))

In [38]:
###########################
#### Print the results ####
###########################

print("")
print("")
#Get the number of elements
long= len(df.index)
#First let separate the success form failure, to make it easier to plot
x1_success=[]
x2_success=[]
x1_failure=[]
x2_failure=[]

for pos in range(long):
    elem=df.iloc[pos]
    if(elem['y']==1):
        x1_success.append(elem['x1'])
        x2_success.append(elem['x2'])
    else:
        x1_failure.append(elem['x1'])
        x2_failure.append(elem['x2'])

if(accuracy!=100):
    #Accuracy have not reach 100%.
    print("We were unable to converge to a set of weights that correctly classifies the data. These data may not be linearly separable.")
    success=plt.scatter(x1_success, x2_success, c="b", marker="s",label="success",alpha=0.75)
    failure=plt.scatter(x1_failure, x2_failure, c="g", marker="^",label="failure",alpha=0.75)

    plt.legend()
    plt.xlim(-0.5,11.5)
    plt.ylim(-0.5,11.5)
    plt.show()
else:
    #We print the Weights that classifies the data
    print("Weights[w0,w1,w2]:" + str(w))
    print("Approximated target function: " + str(w[0]) + " + " + str(w[1]) + " * x1 + " + str(w[2]) + " * x2 = 0")
    
    #Also sow the evolution of the acurracy
    plt.plot(historic_Acurracy, label="Acurracy")
    plt.legend()
    plt.show()
    
    
    #Lets see the data frame
    #Create points to draw the line
    #Point 1: 
    p1x=0
    p1y=( - w[1]*p1x - w[0])/w[2]
    #Point 2:
    p2x=10
    p2y=( - w[1]*p2x - w[0])/w[2]

    success=plt.scatter(x1_success, x2_success, c="b", marker="s",label="success",alpha=0.75)
    failure=plt.scatter(x1_failure, x2_failure, c="g", marker="^",label="failure",alpha=0.75)
    line=plt.plot([p1x,p2x],[p1y,p2y],c="r", label="Prediction Line")

    plt.legend()
    plt.xlim(-0.5,11.5)
    plt.ylim(-0.5,11.5)
    plt.show()